# Calcul atteinte objectifs AMOén

## Variables

In [391]:
import os
import datetime
import numpy as np
import pandas as pd

os.environ['USE_ARROW_extension'] = '1'

## Météo
DJ_REF_ANNUELS = 3260.539010836340
DJ_TEMPERATURE_REFERENCE = 20

## Agents énergétiques conversion
CONVERSION_MAZOUT_KG_MJ = 44.8
CONVERSION_MAZOUT_LITRES_MJ = 44.8 * 0.84
CONVERSION_MAZOUT_KWH_MJ = 3.6
CONVERSION_GAZ_NATUREL_M3_MJ = 38.5
CONVERSION_GAZ_NATUREL_KWH_MJ = 3.6
CONVERSION_BOIS_BUCHES_DUR_STERE_MJ = 7960
CONVERSION_BOIS_BUCHES_TENDRE_STERE_MJ = 5572
CONVERSION_BOIS_BUCHES_TENDRE_KWH_MJ = 3.6
CONVERSION_PELLETS_M3_MJ = 13332
CONVERSION_PELLETS_KG_MJ = 20.2
CONVERSION_PELLETS_KWH_MJ = 3.6
CONVERSION_PLAQUETTES_M3_MJ = 200*20
CONVERSION_PLAQUETTES_KWH_MJ = 3.6*13.1/11.6
CONVERSION_CAD_KWH_MJ = 3.6
CONVERSION_ELECTRICITE_PAC_KWH_MJ = 3.6
CONVERSION_ELECTRICITE_DIRECTE_KWH_MJ = 3.6
CONVERSION_AUTRE_KWH_MJ = 3.6

## Agent énergétique facteur de pondération
FACTEUR_PONDERATION_MAZOUT = 1
FACTEUR_PONDERATION_GAZ_NATUREL = 1
FACTEUR_PONDERATION_BOIS_BUCHES_DUR = 1
FACTEUR_PONDERATION_BOIS_BUCHES_TENDRE = 1
FACTEUR_PONDERATION_PELLETS = 1
FACTEUR_PONDERATION_PLAQUETTES = 1
FACTEUR_PONDERATION_CAD = 1
FACTEUR_PONDERATION_ELECTRICITE_PAC = 2
FACTEUR_PONDERATION_ELECTRICITE_DIRECTE = 2
FACTEUR_PONDERATION_AUTRE = 1

# Variable pour la mise a jour de la météo
last_update_time_meteo = datetime.datetime(2021, 1, 1)



## Météo

In [392]:

# Météo
def get_meteo_data():
    global DJ_TEMPERATURE_REFERENCE
    # Mise à jour des données météo de manière journalière
    df_meteo_tre200d0_historique = pd.read_csv("https://data.geo.admin.ch/ch.meteoschweiz.klima/nbcn-tageswerte/nbcn-daily_GVE_previous.csv", sep=';', encoding='latin1', parse_dates=['date'])
    df_meteo_tre200d0_annee_cours = pd.read_csv("https://data.geo.admin.ch/ch.meteoschweiz.klima/nbcn-tageswerte/nbcn-daily_GVE_current.csv", sep=';', encoding='latin1', parse_dates=['date'])
    df_meteo_tre200d0 = pd.concat([df_meteo_tre200d0_historique, df_meteo_tre200d0_annee_cours])
    df_meteo_tre200d0.rename(columns={"station/location": "stn",
                        'date': 'time'}, inplace=True)
    df_meteo_tre200d0 = df_meteo_tre200d0[['stn', 'time', 'tre200d0']]
    df_meteo_tre200d0 = df_meteo_tre200d0[df_meteo_tre200d0['time'] >= '2015-01-01']
    df_meteo_tre200d0.drop_duplicates(inplace=True)
    # replace values with "-" with nan and drop them
    df_meteo_tre200d0.replace("-", pd.NA, inplace=True)
    df_meteo_tre200d0.dropna(inplace=True)
    df_meteo_tre200d0.sort_values(by="time", inplace=True)
    df_meteo_tre200d0.reset_index(drop=True, inplace=True)
    # Calculs pour DJ
    df_meteo_tre200d0['tre200d0'] = df_meteo_tre200d0['tre200d0'].astype(float)
    df_meteo_tre200d0['mois'] = df_meteo_tre200d0['time'].dt.month
    df_meteo_tre200d0['saison_chauffe'] = np.where(df_meteo_tre200d0['mois'].isin([9, 10, 11, 12, 1, 2, 3, 4, 5]), 1, 0)
    df_meteo_tre200d0['tre200d0_sous_16'] = np.where(df_meteo_tre200d0['tre200d0'] <= 16, 1, 0)
    df_meteo_tre200d0['DJ_theta0_16'] = df_meteo_tre200d0['saison_chauffe'] * df_meteo_tre200d0['tre200d0_sous_16'] * (DJ_TEMPERATURE_REFERENCE - df_meteo_tre200d0['tre200d0'])

    return df_meteo_tre200d0

In [393]:
# Calcul des degrés-jours
def calcul_dj_periode(df_meteo_tre200d0, periode_start, periode_end):
    dj_periode = df_meteo_tre200d0[(df_meteo_tre200d0['time'] >= periode_start) & (df_meteo_tre200d0['time'] <= periode_end)]['DJ_theta0_16'].sum()
    return dj_periode

## Inputs

In [394]:
# # SRE
# sre_renovation_m2 = 0

# # SRE pourcentage affectations
# sre_pourcentage_habitat_collectif = 0.0
# sre_pourcentage_habitat_individuel = 0.0
# sre_pourcentage_administration = 0.0
# sre_pourcentage_ecoles = 0.0
# sre_pourcentage_commerce = 0.0
# sre_pourcentage_restauration = 0.0
# sre_pourcentage_lieux_de_rassemblement = 0.0
# sre_pourcentage_hopitaux = 0.0
# sre_pourcentage_industrie = 0.0
# sre_pourcentage_depots = 0.0
# sre_pourcentage_installations_sportives = 0.0
# sre_pourcentage_piscines_couvertes = 0.0

# # Agents énergétiques
# agent_energetique_ef_mazout_kg = 0.0
# agent_energetique_ef_mazout_litres = 0.0
# agent_energetique_ef_mazout_kwh = 0.0
# agent_energetique_ef_gaz_naturel_m3 = 0.0
# agent_energetique_ef_gaz_naturel_kwh = 0.0
# agent_energetique_ef_bois_buches_dur_stere = 0.0
# agent_energetique_ef_bois_buches_tendre_stere = 0.0
# agent_energetique_ef_bois_buches_tendre_kwh = 0.0
# agent_energetique_ef_pellets_m3 = 0.0
# agent_energetique_ef_pellets_kg = 0.0
# agent_energetique_ef_pellets_kwh = 0.0
# agent_energetique_ef_plaquettes_m3 = 0.0
# agent_energetique_ef_plaquettes_kwh = 0.0
# agent_energetique_ef_cad_kwh = 0.0
# agent_energetique_ef_electricite_pac_kwh = 0.0
# agent_energetique_ef_electricite_directe_kwh = 0.0
# agent_energetique_ef_autre_kwh = 0.0

# # Météo début/fin
# periode_start = datetime.date(2023,1,1)
# periode_end = datetime.date(2023,12,31)

# # Autres données
# ef_avant_corr_kwh_m2 = 0.0
# ef_objectif_pondere_kwh_m2 = 0.0

# # Répartition énergie finale
# repartition_energie_finale_partie_renovee_chauffage = 0.0
# repartition_energie_finale_partie_renovee_ecs = 0.0
# repartition_energie_finale_partie_surelevee_chauffage = 0.0
# repartition_energie_finale_partie_surelevee_ecs = 0.0


## Calcul atteinte performance AMOén

In [395]:
def calcul_atteinte_objectifs_amoen(sre_renovation_m2,
                                    sre_pourcentage_habitat_collectif,
                                    sre_pourcentage_habitat_individuel,
                                    sre_pourcentage_administration,
                                    sre_pourcentage_ecoles,
                                    sre_pourcentage_commerce,
                                    sre_pourcentage_restauration,
                                    sre_pourcentage_lieux_de_rassemblement,
                                    sre_pourcentage_hopitaux,
                                    sre_pourcentage_industrie,
                                    sre_pourcentage_depots,
                                    sre_pourcentage_installations_sportives,
                                    sre_pourcentage_piscines_couvertes,
                                    agent_energetique_ef_mazout_kg,
                                    agent_energetique_ef_mazout_litres,
                                    agent_energetique_ef_mazout_kwh,
                                    agent_energetique_ef_gaz_naturel_m3,
                                    agent_energetique_ef_gaz_naturel_kwh,
                                    agent_energetique_ef_bois_buches_dur_stere,
                                    agent_energetique_ef_bois_buches_tendre_stere,
                                    agent_energetique_ef_bois_buches_tendre_kwh,
                                    agent_energetique_ef_pellets_m3,
                                    agent_energetique_ef_pellets_kg,
                                    agent_energetique_ef_pellets_kwh,
                                    agent_energetique_ef_plaquettes_m3,
                                    agent_energetique_ef_plaquettes_kwh,
                                    agent_energetique_ef_cad_kwh,
                                    agent_energetique_ef_electricite_pac_kwh,
                                    agent_energetique_ef_electricite_directe_kwh,
                                    agent_energetique_ef_autre_kwh,
                                    periode_start,
                                    periode_end,
                                    ef_avant_corr_kwh_m2,
                                    ef_objectif_pondere_kwh_m2,
                                    repartition_energie_finale_partie_renovee_chauffage,
                                    repartition_energie_finale_partie_renovee_ecs,
                                    repartition_energie_finale_partie_surelevee_chauffage,
                                    repartition_energie_finale_partie_surelevee_ecs,
                                    df_meteo_tre200d0):
    # somme des pourcentages d'affectations doit être égale à 100%
    sre_pourcentage_affectations_somme_avertisement = float(sre_pourcentage_habitat_collectif) + \
        float(sre_pourcentage_habitat_individuel) + \
        float(sre_pourcentage_administration) + \
        float(sre_pourcentage_ecoles) + \
        float(sre_pourcentage_commerce) + \
        float(sre_pourcentage_restauration) + \
        float(sre_pourcentage_lieux_de_rassemblement) + \
        float(sre_pourcentage_hopitaux) + \
        float(sre_pourcentage_industrie) + \
        float(sre_pourcentage_depots) + \
        float(sre_pourcentage_installations_sportives) + \
        float(sre_pourcentage_piscines_couvertes)
    assert(sre_pourcentage_affectations_somme_avertisement == 100.0)

    # somme des agents énergétiques doit être supérieur à 0
    agent_energetique_ef_somme_avertissement = agent_energetique_ef_mazout_kg + \
        agent_energetique_ef_mazout_litres + \
        agent_energetique_ef_mazout_kwh + \
        agent_energetique_ef_gaz_naturel_m3 + \
        agent_energetique_ef_gaz_naturel_kwh + \
        agent_energetique_ef_bois_buches_dur_stere + \
        agent_energetique_ef_bois_buches_tendre_stere + \
        agent_energetique_ef_bois_buches_tendre_kwh + \
        agent_energetique_ef_pellets_m3 + \
        agent_energetique_ef_pellets_kg + \
        agent_energetique_ef_pellets_kwh + \
        agent_energetique_ef_plaquettes_m3 + \
        agent_energetique_ef_plaquettes_kwh + \
        agent_energetique_ef_cad_kwh + \
        agent_energetique_ef_electricite_pac_kwh + \
        agent_energetique_ef_electricite_directe_kwh + \
        agent_energetique_ef_autre_kwh
    assert(agent_energetique_ef_somme_avertissement > 0)

    # Météo pour la période
    periode_nb_jours = float((periode_end - periode_start).days + 1)
    periode_start = pd.to_datetime(periode_start)
    periode_end = pd.to_datetime(periode_end)
    assert(periode_nb_jours > 0)
    assert(periode_start < periode_end)

    # Autre données
    delta_ef_visee_kwh_m2 = float(ef_avant_corr_kwh_m2) - float(ef_objectif_pondere_kwh_m2)

    # Répartition énergie finale doit être égale à 100%
    repartition_ef_somme_avertissement = repartition_energie_finale_partie_renovee_chauffage + \
        repartition_energie_finale_partie_renovee_ecs +\
        repartition_energie_finale_partie_surelevee_chauffage + \
        repartition_energie_finale_partie_surelevee_ecs
    assert(repartition_ef_somme_avertissement == 100)

    # C91 → Part EF pour partie rénové (Chauffage + ECS)
    repartition_energie_finale_partie_renovee_somme = repartition_energie_finale_partie_renovee_chauffage + repartition_energie_finale_partie_renovee_ecs

    # C92 → Est. ECS/ECS annuelle
    estimation_ecs_annuel = periode_nb_jours/365

    # C93 → Est. Chauffage/Chauffage annuel prévisible
    dj_periode = calcul_dj_periode(df_meteo_tre200d0, periode_start, periode_end)
    dj_periode = float(dj_periode)
    estimation_part_chauffage_periode_sur_annuel = dj_periode / DJ_REF_ANNUELS
    estimation_part_chauffage_periode_sur_annuel = float(estimation_part_chauffage_periode_sur_annuel)

    # C94 → Est. EF période / EF année
    estimation_energie_finale_periode_sur_annuel = (estimation_ecs_annuel * (repartition_energie_finale_partie_renovee_ecs + repartition_energie_finale_partie_surelevee_ecs)) +\
                                        (estimation_part_chauffage_periode_sur_annuel * (repartition_energie_finale_partie_renovee_chauffage + repartition_energie_finale_partie_surelevee_chauffage))

    # C95 → Est. Part ECS période comptage
    try:
        if estimation_energie_finale_periode_sur_annuel != 0:
            part_ecs_periode_comptage = (estimation_ecs_annuel * (repartition_energie_finale_partie_renovee_ecs + \
                repartition_energie_finale_partie_surelevee_ecs)) / estimation_energie_finale_periode_sur_annuel
        else:
            part_ecs_periode_comptage = 0.0
    except ZeroDivisionError:
        part_ecs_periode_comptage = 0.0


    # C96 → Est. Part Chauffage période comptage
    try:
        if estimation_energie_finale_periode_sur_annuel != 0:
            part_chauffage_periode_comptage = (estimation_part_chauffage_periode_sur_annuel * \
                (repartition_energie_finale_partie_renovee_chauffage + repartition_energie_finale_partie_surelevee_chauffage)) / estimation_energie_finale_periode_sur_annuel
        else:
            part_chauffage_periode_comptage = 0.0
    except ZeroDivisionError:
        part_chauffage_periode_comptage = 0.0

    # C97 → correction ECS = 365/nb jour comptage
    correction_ecs = 365/periode_nb_jours

    # C98 → Energie finale indiqué par le(s) compteur(s)
    agent_energetique_ef_mazout_somme_mj = (agent_energetique_ef_mazout_kg * CONVERSION_MAZOUT_KG_MJ + agent_energetique_ef_mazout_litres * CONVERSION_MAZOUT_LITRES_MJ + agent_energetique_ef_mazout_kwh * CONVERSION_MAZOUT_KWH_MJ)
    agent_energetique_ef_gaz_naturel_somme_mj = (agent_energetique_ef_gaz_naturel_m3 * CONVERSION_GAZ_NATUREL_M3_MJ + agent_energetique_ef_gaz_naturel_kwh * CONVERSION_GAZ_NATUREL_KWH_MJ)
    agent_energetique_ef_bois_buches_dur_somme_mj = (agent_energetique_ef_bois_buches_dur_stere * CONVERSION_BOIS_BUCHES_DUR_STERE_MJ)
    agent_energetique_ef_bois_buches_tendre_somme_mj = (agent_energetique_ef_bois_buches_tendre_stere * CONVERSION_BOIS_BUCHES_TENDRE_STERE_MJ + agent_energetique_ef_bois_buches_tendre_kwh * CONVERSION_BOIS_BUCHES_TENDRE_KWH_MJ)
    agent_energetique_ef_pellets_somme_mj = (agent_energetique_ef_pellets_m3 * CONVERSION_PELLETS_M3_MJ + agent_energetique_ef_pellets_kg * CONVERSION_PELLETS_KG_MJ + agent_energetique_ef_pellets_kwh * CONVERSION_PELLETS_KWH_MJ)
    agent_energetique_ef_plaquettes_somme_mj = (agent_energetique_ef_plaquettes_m3 * CONVERSION_PLAQUETTES_M3_MJ + agent_energetique_ef_plaquettes_kwh * CONVERSION_PLAQUETTES_KWH_MJ)
    agent_energetique_ef_cad_somme_mj = (agent_energetique_ef_cad_kwh * CONVERSION_CAD_KWH_MJ)
    agent_energetique_ef_electricite_pac_somme_mj = (agent_energetique_ef_electricite_pac_kwh * CONVERSION_ELECTRICITE_PAC_KWH_MJ)
    agent_energetique_ef_electricite_directe_somme_mj = (agent_energetique_ef_electricite_directe_kwh * CONVERSION_ELECTRICITE_DIRECTE_KWH_MJ)
    agent_energetique_ef_autre_somme_mj = (agent_energetique_ef_autre_kwh * CONVERSION_AUTRE_KWH_MJ)

    agent_energetique_ef_somme_kwh = (agent_energetique_ef_mazout_somme_mj + \
                                agent_energetique_ef_gaz_naturel_somme_mj + \
                                    agent_energetique_ef_bois_buches_dur_somme_mj + \
                                    agent_energetique_ef_bois_buches_tendre_somme_mj + \
                                    agent_energetique_ef_pellets_somme_mj + \
                                    agent_energetique_ef_plaquettes_somme_mj + \
                                    agent_energetique_ef_cad_somme_mj +\
                                    agent_energetique_ef_electricite_pac_somme_mj + \
                                    agent_energetique_ef_electricite_directe_somme_mj + \
                                    agent_energetique_ef_autre_somme_mj) / 3.6
    assert(agent_energetique_ef_somme_kwh > 0)

    # C99 → Methodo_Bww → Part de ECS en énergie finale sur la période
    methodo_b_ww_kwh = (agent_energetique_ef_somme_kwh) * part_ecs_periode_comptage

    # C100 → Methodo_Eww
    try:
        if sre_renovation_m2 != 0 and periode_nb_jours != 0:
            methodo_e_ww_kwh = (methodo_b_ww_kwh / sre_renovation_m2) * (365 / periode_nb_jours)
        else:
            methodo_e_ww_kwh = 0.0
    except ZeroDivisionError:
        methodo_e_ww_kwh = 0.0

    # C101 → DJ ref annuels
    DJ_REF_ANNUELS

    # C102 → DJ période comptage
    dj_periode

    # C103 → Methodo_Bh → Part de chauffage en énergie finale sur la période
    methodo_b_h_kwh = agent_energetique_ef_somme_kwh * part_chauffage_periode_comptage

    # C104 → Methodo_Eh
    try:
        if sre_renovation_m2 != 0 and dj_periode != 0:
            methodo_e_h_kwh = (methodo_b_h_kwh / sre_renovation_m2) * (DJ_REF_ANNUELS / dj_periode)
        else:
            methodo_e_h_kwh = 0.0
    except ZeroDivisionError:
        methodo_e_h_kwh = 0.0

    # C105 → Ef,après,corr → Methodo_Eww + Methodo_Eh
    energie_finale_apres_travaux_climatiquement_corrigee_inclus_surelevation_kwh_m2 = methodo_e_ww_kwh + methodo_e_h_kwh

    # C106 → Part de l'énergie finale théorique dédiée à la partie rénovée (ECS+Ch.)
    repartition_energie_finale_partie_renovee_somme

    # C107 → Ef,après,corr,rénové →Total en énergie finale (Eww+Eh) pour la partie rénovée
    energie_finale_apres_travaux_climatiquement_corrigee_renovee_kwh_m2 = energie_finale_apres_travaux_climatiquement_corrigee_inclus_surelevation_kwh_m2 * (repartition_energie_finale_partie_renovee_somme / 100)

    # C108 → fp → facteur de pondération moyen
    try:
        if agent_energetique_ef_somme_kwh:
            facteur_ponderation_moyen = (agent_energetique_ef_mazout_somme_mj * FACTEUR_PONDERATION_MAZOUT + \
                                agent_energetique_ef_gaz_naturel_somme_mj * FACTEUR_PONDERATION_GAZ_NATUREL + \
                                agent_energetique_ef_bois_buches_dur_somme_mj * FACTEUR_PONDERATION_BOIS_BUCHES_DUR + \
                                agent_energetique_ef_bois_buches_tendre_somme_mj * FACTEUR_PONDERATION_BOIS_BUCHES_TENDRE + \
                                agent_energetique_ef_pellets_somme_mj * FACTEUR_PONDERATION_PELLETS + \
                                agent_energetique_ef_plaquettes_somme_mj * FACTEUR_PONDERATION_PLAQUETTES + \
                                agent_energetique_ef_cad_somme_mj * FACTEUR_PONDERATION_CAD + \
                                agent_energetique_ef_electricite_pac_somme_mj * FACTEUR_PONDERATION_ELECTRICITE_PAC + \
                                agent_energetique_ef_electricite_directe_somme_mj * FACTEUR_PONDERATION_ELECTRICITE_DIRECTE + \
                                agent_energetique_ef_autre_somme_mj * FACTEUR_PONDERATION_AUTRE) / (agent_energetique_ef_somme_kwh * 3.6)
        else:
            facteur_ponderation_moyen = 0
    except ZeroDivisionError:
        facteur_ponderation_moyen = 0

    # C109 → Methodo_Eww*fp
    methodo_e_ww_renovee_pondere_kwh_m2 = methodo_e_ww_kwh * facteur_ponderation_moyen * (repartition_energie_finale_partie_renovee_somme / 100)

    # C110 → Methodo_Eh*fp
    methodo_e_h_renovee_pondere_kwh_m2 = methodo_e_h_kwh * facteur_ponderation_moyen * (repartition_energie_finale_partie_renovee_somme / 100)

    # C113 → Ef,après,corr,rénové*fp
    energie_finale_apres_travaux_climatiquement_corrigee_renovee_pondere_kwh_m2 = energie_finale_apres_travaux_climatiquement_corrigee_renovee_kwh_m2 * facteur_ponderation_moyen

    # C114 → Ef,après,corr,rénové*fp
    energie_finale_apres_travaux_climatiquement_corrigee_renovee_pondere_MJ_m2 = energie_finale_apres_travaux_climatiquement_corrigee_renovee_pondere_kwh_m2 * 3.6

    return delta_ef_visee_kwh_m2,\
        estimation_ecs_annuel,\
        estimation_part_chauffage_periode_sur_annuel,\
        estimation_energie_finale_periode_sur_annuel,\
        part_ecs_periode_comptage,\
        part_chauffage_periode_comptage,\
        correction_ecs,\
        agent_energetique_ef_somme_kwh,\
        methodo_b_ww_kwh,\
        methodo_e_ww_kwh,\
        DJ_REF_ANNUELS,\
        dj_periode,\
        methodo_b_h_kwh,\
        methodo_e_h_kwh,\
        energie_finale_apres_travaux_climatiquement_corrigee_inclus_surelevation_kwh_m2,\
        repartition_energie_finale_partie_renovee_somme,\
        energie_finale_apres_travaux_climatiquement_corrigee_renovee_kwh_m2,\
        facteur_ponderation_moyen,\
        methodo_e_ww_renovee_pondere_kwh_m2,\
        methodo_e_h_renovee_pondere_kwh_m2,\
        energie_finale_apres_travaux_climatiquement_corrigee_renovee_pondere_kwh_m2,\
        energie_finale_apres_travaux_climatiquement_corrigee_renovee_pondere_MJ_m2


## Test unit
### Sans surélévation 1

In [396]:
# SRE
sre_renovation_m2 = 1000

# SRE pourcentage affectations
sre_pourcentage_habitat_collectif = 100.0
sre_pourcentage_habitat_individuel = 0.0
sre_pourcentage_administration = 0.0
sre_pourcentage_ecoles = 0.0
sre_pourcentage_commerce = 0.0
sre_pourcentage_restauration = 0.0
sre_pourcentage_lieux_de_rassemblement = 0.0
sre_pourcentage_hopitaux = 0.0
sre_pourcentage_industrie = 0.0
sre_pourcentage_depots = 0.0
sre_pourcentage_installations_sportives = 0.0
sre_pourcentage_piscines_couvertes = 0.0

# Agents énergétiques
agent_energetique_ef_mazout_kg = 0.0
agent_energetique_ef_mazout_litres = 0.0
agent_energetique_ef_mazout_kwh = 0.0
agent_energetique_ef_gaz_naturel_m3 = 0.0
agent_energetique_ef_gaz_naturel_kwh = 10000
agent_energetique_ef_bois_buches_dur_stere = 0.0
agent_energetique_ef_bois_buches_tendre_stere = 0.0
agent_energetique_ef_bois_buches_tendre_kwh = 0.0
agent_energetique_ef_pellets_m3 = 0.0
agent_energetique_ef_pellets_kg = 0.0
agent_energetique_ef_pellets_kwh = 0.0
agent_energetique_ef_plaquettes_m3 = 0.0
agent_energetique_ef_plaquettes_kwh = 0.0
agent_energetique_ef_cad_kwh = 0.0
agent_energetique_ef_electricite_pac_kwh = 0.0
agent_energetique_ef_electricite_directe_kwh = 0.0
agent_energetique_ef_autre_kwh = 0.0

# Météo début/fin
periode_start = datetime.date(2022,6,1)
periode_end = datetime.date(2022,12,31)

# Autres données
ef_avant_corr_kwh_m2 = 138.888888888889
ef_objectif_pondere_kwh_m2 = 25.846153846154

# Répartition énergie finale
repartition_energie_finale_partie_renovee_chauffage = 61.904761904762
repartition_energie_finale_partie_renovee_ecs = 38.095238095238
repartition_energie_finale_partie_surelevee_chauffage = 0.0
repartition_energie_finale_partie_surelevee_ecs = 0.0

df_meteo_tre200d0 = get_meteo_data()

delta_ef_visee_kwh_m2,\
    estimation_ecs_annuel,\
    estimation_part_chauffage_periode_sur_annuel,\
    estimation_energie_finale_periode_sur_annuel,\
    part_ecs_periode_comptage,\
    part_chauffage_periode_comptage,\
    correction_ecs,\
    agent_energetique_ef_somme_kwh,\
    methodo_b_ww_kwh,\
    methodo_e_ww_kwh,\
    DJ_REF_ANNUELS,\
    dj_periode,\
    methodo_b_h_kwh,\
    methodo_e_h_kwh,\
    energie_finale_apres_travaux_climatiquement_corrigee_inclus_surelevation_kwh_m2,\
    repartition_energie_finale_partie_renovee_somme,\
    energie_finale_apres_travaux_climatiquement_corrigee_renovee_kwh_m2,\
    facteur_ponderation_moyen,\
    methodo_e_ww_renovee_pondere_kwh_m2,\
    methodo_e_h_renovee_pondere_kwh_m2,\
    energie_finale_apres_travaux_climatiquement_corrigee_renovee_pondere_kwh_m2,\
    energie_finale_apres_travaux_climatiquement_corrigee_renovee_pondere_MJ_m2 = calcul_atteinte_objectifs_amoen(sre_renovation_m2=sre_renovation_m2,\
                                                                                                                sre_pourcentage_habitat_collectif=sre_pourcentage_habitat_collectif,\
                                                                                                                sre_pourcentage_habitat_individuel=sre_pourcentage_habitat_individuel,\
                                                                                                                sre_pourcentage_administration=sre_pourcentage_administration,\
                                                                                                                sre_pourcentage_ecoles=sre_pourcentage_ecoles,\
                                                                                                                sre_pourcentage_commerce=sre_pourcentage_commerce,\
                                                                                                                sre_pourcentage_restauration=sre_pourcentage_restauration,\
                                                                                                                sre_pourcentage_lieux_de_rassemblement=sre_pourcentage_lieux_de_rassemblement,\
                                                                                                                sre_pourcentage_hopitaux=sre_pourcentage_hopitaux,\
                                                                                                                sre_pourcentage_industrie=sre_pourcentage_industrie,\
                                                                                                                sre_pourcentage_depots=sre_pourcentage_depots,\
                                                                                                                sre_pourcentage_installations_sportives=sre_pourcentage_installations_sportives,\
                                                                                                                sre_pourcentage_piscines_couvertes=sre_pourcentage_piscines_couvertes,\
                                                                                                                agent_energetique_ef_mazout_kg=agent_energetique_ef_mazout_kg,\
                                                                                                                agent_energetique_ef_mazout_litres=agent_energetique_ef_mazout_litres,\
                                                                                                                agent_energetique_ef_mazout_kwh=agent_energetique_ef_mazout_kwh,\
                                                                                                                agent_energetique_ef_gaz_naturel_m3=agent_energetique_ef_gaz_naturel_m3,\
                                                                                                                agent_energetique_ef_gaz_naturel_kwh=agent_energetique_ef_gaz_naturel_kwh,\
                                                                                                                agent_energetique_ef_bois_buches_dur_stere=agent_energetique_ef_bois_buches_dur_stere,\
                                                                                                                agent_energetique_ef_bois_buches_tendre_stere=agent_energetique_ef_bois_buches_tendre_stere,\
                                                                                                                agent_energetique_ef_bois_buches_tendre_kwh=agent_energetique_ef_bois_buches_tendre_kwh,\
                                                                                                                agent_energetique_ef_pellets_m3=agent_energetique_ef_pellets_m3,\
                                                                                                                agent_energetique_ef_pellets_kg=agent_energetique_ef_pellets_kg,\
                                                                                                                agent_energetique_ef_pellets_kwh=agent_energetique_ef_pellets_kwh,\
                                                                                                                agent_energetique_ef_plaquettes_m3=agent_energetique_ef_plaquettes_m3,\
                                                                                                                agent_energetique_ef_plaquettes_kwh=agent_energetique_ef_plaquettes_kwh,\
                                                                                                                agent_energetique_ef_cad_kwh=agent_energetique_ef_cad_kwh,\
                                                                                                                agent_energetique_ef_electricite_pac_kwh=agent_energetique_ef_electricite_pac_kwh,\
                                                                                                                agent_energetique_ef_electricite_directe_kwh=agent_energetique_ef_electricite_directe_kwh,\
                                                                                                                agent_energetique_ef_autre_kwh=agent_energetique_ef_autre_kwh,\
                                                                                                                periode_start=periode_start,\
                                                                                                                periode_end=periode_end,\
                                                                                                                ef_avant_corr_kwh_m2=ef_avant_corr_kwh_m2,\
                                                                                                                ef_objectif_pondere_kwh_m2=ef_objectif_pondere_kwh_m2,\
                                                                                                                repartition_energie_finale_partie_renovee_chauffage=repartition_energie_finale_partie_renovee_chauffage,\
                                                                                                                repartition_energie_finale_partie_renovee_ecs=repartition_energie_finale_partie_renovee_ecs,\
                                                                                                                repartition_energie_finale_partie_surelevee_chauffage=repartition_energie_finale_partie_surelevee_chauffage,\
                                                                                                                repartition_energie_finale_partie_surelevee_ecs=repartition_energie_finale_partie_surelevee_ecs,\
                                                                                                                df_meteo_tre200d0=df_meteo_tre200d0)

assert(delta_ef_visee_kwh_m2 == 113.042735042735)
# C92 → Est. ECS/ECS annuelle
assert(round(estimation_ecs_annuel, 10) == 0.5863013699)
# C93 → Est. Chauffage/Chauffage annuel prévisible
assert(round(estimation_part_chauffage_periode_sur_annuel, 10) == 0.3292707115)
# C94 → Est. EF période / EF année
assert(round(estimation_energie_finale_periode_sur_annuel, 10) == 42.7187152787)
# C95 → Est. Part ECS période comptage
assert(round(part_ecs_periode_comptage, 10) == 0.5228455522)
# C96 → Est. Part Chauffage période comptage
assert(round(part_chauffage_periode_comptage, 10) == 0.4771544478)
# C97 → correction ECS = 365/nb jour comptage
assert(round(correction_ecs, 10) == 1.7056074766)
# C98 → Energie finale indiqué par le(s) compteur(s)
assert(round(agent_energetique_ef_somme_kwh, 10) == 10000.0)
# C99 → Methodo_Bww → Part de ECS en énergie finale sur la période
assert(round(methodo_b_ww_kwh, 10) == 5228.4555223168)
# C100 → Methodo_Eww
assert(round(methodo_e_ww_kwh, 10) == 8.9176928301)
# C101 → DJ ref annuels
assert(round(DJ_REF_ANNUELS, 10) == 3260.5390108363)
# C102 → DJ période comptage
assert(round(dj_periode, 10) == 1073.6)
# C103 → Methodo_Bh → Part de chauffage en énergie finale sur la période
assert(round(methodo_b_h_kwh, 10) == 4771.5444776832)
# C104 → Methodo_Eh
assert(round(methodo_e_h_kwh, 10) == 14.4912508489)
# C105 → Ef,après,corr → Methodo_Eww + Methodo_Eh
assert(round(energie_finale_apres_travaux_climatiquement_corrigee_inclus_surelevation_kwh_m2, 10) == 23.4089436791)
# C106 → Part de l'énergie finale théorique dédiée à la partie rénovée (ECS+Ch.)
assert(round(repartition_energie_finale_partie_renovee_somme, 12) == 100.0)
# C107 → Ef,après,corr,rénové →Total en énergie finale (Eww+Eh) pour la partie rénovée
assert(round(energie_finale_apres_travaux_climatiquement_corrigee_renovee_kwh_m2, 10) == 23.4089436791)
# C108 → fp → facteur de pondération moyen
assert(round(facteur_ponderation_moyen, 12) == 1.0)
# C109 → Methodo_Eww*fp
assert(round(methodo_e_ww_renovee_pondere_kwh_m2, 10) == 8.9176928301)
# C110 → Methodo_Eh*fp
assert(round(methodo_e_h_renovee_pondere_kwh_m2, 10) == 14.4912508489)
# C113 → Ef,après,corr,rénové*fp
assert(round(energie_finale_apres_travaux_climatiquement_corrigee_renovee_pondere_kwh_m2, 10) == 23.4089436791)
# C114 → Ef,après,corr,rénové*fp
assert(round(energie_finale_apres_travaux_climatiquement_corrigee_renovee_pondere_MJ_m2, 10) == 84.2721972446)



### Sans surélévation 2

In [397]:
# SRE
sre_renovation_m2 = 1500

# SRE pourcentage affectations
sre_pourcentage_habitat_collectif = 80.0
sre_pourcentage_habitat_individuel = 0.0
sre_pourcentage_administration = 20.0
sre_pourcentage_ecoles = 0.0
sre_pourcentage_commerce = 0.0
sre_pourcentage_restauration = 0.0
sre_pourcentage_lieux_de_rassemblement = 0.0
sre_pourcentage_hopitaux = 0.0
sre_pourcentage_industrie = 0.0
sre_pourcentage_depots = 0.0
sre_pourcentage_installations_sportives = 0.0
sre_pourcentage_piscines_couvertes = 0.0

# Agents énergétiques
agent_energetique_ef_mazout_kg = 0.0
agent_energetique_ef_mazout_litres = 500
agent_energetique_ef_mazout_kwh = 0.0
agent_energetique_ef_gaz_naturel_m3 = 0.0
agent_energetique_ef_gaz_naturel_kwh = 10000
agent_energetique_ef_bois_buches_dur_stere = 0.0
agent_energetique_ef_bois_buches_tendre_stere = 0.0
agent_energetique_ef_bois_buches_tendre_kwh = 0.0
agent_energetique_ef_pellets_m3 = 0.0
agent_energetique_ef_pellets_kg = 0.0
agent_energetique_ef_pellets_kwh = 0.0
agent_energetique_ef_plaquettes_m3 = 0.0
agent_energetique_ef_plaquettes_kwh = 0.0
agent_energetique_ef_cad_kwh = 10000
agent_energetique_ef_electricite_pac_kwh = 10000
agent_energetique_ef_electricite_directe_kwh = 0.0
agent_energetique_ef_autre_kwh = 0.0

# Météo début/fin
periode_start = datetime.date(2022,1,1)
periode_end = datetime.date(2022,12,31)

# Autres données
ef_avant_corr_kwh_m2 = 138.888888888889
ef_objectif_pondere_kwh_m2 = 25.846153846154

# Répartition énergie finale
repartition_energie_finale_partie_renovee_chauffage = 61.904761904762
repartition_energie_finale_partie_renovee_ecs = 38.095238095238
repartition_energie_finale_partie_surelevee_chauffage = 0.0
repartition_energie_finale_partie_surelevee_ecs = 0.0

df_meteo_tre200d0 = get_meteo_data()

delta_ef_visee_kwh_m2,\
    estimation_ecs_annuel,\
    estimation_part_chauffage_periode_sur_annuel,\
    estimation_energie_finale_periode_sur_annuel,\
    part_ecs_periode_comptage,\
    part_chauffage_periode_comptage,\
    correction_ecs,\
    agent_energetique_ef_somme_kwh,\
    methodo_b_ww_kwh,\
    methodo_e_ww_kwh,\
    DJ_REF_ANNUELS,\
    dj_periode,\
    methodo_b_h_kwh,\
    methodo_e_h_kwh,\
    energie_finale_apres_travaux_climatiquement_corrigee_inclus_surelevation_kwh_m2,\
    repartition_energie_finale_partie_renovee_somme,\
    energie_finale_apres_travaux_climatiquement_corrigee_renovee_kwh_m2,\
    facteur_ponderation_moyen,\
    methodo_e_ww_renovee_pondere_kwh_m2,\
    methodo_e_h_renovee_pondere_kwh_m2,\
    energie_finale_apres_travaux_climatiquement_corrigee_renovee_pondere_kwh_m2,\
    energie_finale_apres_travaux_climatiquement_corrigee_renovee_pondere_MJ_m2 = calcul_atteinte_objectifs_amoen(sre_renovation_m2=sre_renovation_m2,\
                                                                                                                sre_pourcentage_habitat_collectif=sre_pourcentage_habitat_collectif,\
                                                                                                                sre_pourcentage_habitat_individuel=sre_pourcentage_habitat_individuel,\
                                                                                                                sre_pourcentage_administration=sre_pourcentage_administration,\
                                                                                                                sre_pourcentage_ecoles=sre_pourcentage_ecoles,\
                                                                                                                sre_pourcentage_commerce=sre_pourcentage_commerce,\
                                                                                                                sre_pourcentage_restauration=sre_pourcentage_restauration,\
                                                                                                                sre_pourcentage_lieux_de_rassemblement=sre_pourcentage_lieux_de_rassemblement,\
                                                                                                                sre_pourcentage_hopitaux=sre_pourcentage_hopitaux,\
                                                                                                                sre_pourcentage_industrie=sre_pourcentage_industrie,\
                                                                                                                sre_pourcentage_depots=sre_pourcentage_depots,\
                                                                                                                sre_pourcentage_installations_sportives=sre_pourcentage_installations_sportives,\
                                                                                                                sre_pourcentage_piscines_couvertes=sre_pourcentage_piscines_couvertes,\
                                                                                                                agent_energetique_ef_mazout_kg=agent_energetique_ef_mazout_kg,\
                                                                                                                agent_energetique_ef_mazout_litres=agent_energetique_ef_mazout_litres,\
                                                                                                                agent_energetique_ef_mazout_kwh=agent_energetique_ef_mazout_kwh,\
                                                                                                                agent_energetique_ef_gaz_naturel_m3=agent_energetique_ef_gaz_naturel_m3,\
                                                                                                                agent_energetique_ef_gaz_naturel_kwh=agent_energetique_ef_gaz_naturel_kwh,\
                                                                                                                agent_energetique_ef_bois_buches_dur_stere=agent_energetique_ef_bois_buches_dur_stere,\
                                                                                                                agent_energetique_ef_bois_buches_tendre_stere=agent_energetique_ef_bois_buches_tendre_stere,\
                                                                                                                agent_energetique_ef_bois_buches_tendre_kwh=agent_energetique_ef_bois_buches_tendre_kwh,\
                                                                                                                agent_energetique_ef_pellets_m3=agent_energetique_ef_pellets_m3,\
                                                                                                                agent_energetique_ef_pellets_kg=agent_energetique_ef_pellets_kg,\
                                                                                                                agent_energetique_ef_pellets_kwh=agent_energetique_ef_pellets_kwh,\
                                                                                                                agent_energetique_ef_plaquettes_m3=agent_energetique_ef_plaquettes_m3,\
                                                                                                                agent_energetique_ef_plaquettes_kwh=agent_energetique_ef_plaquettes_kwh,\
                                                                                                                agent_energetique_ef_cad_kwh=agent_energetique_ef_cad_kwh,\
                                                                                                                agent_energetique_ef_electricite_pac_kwh=agent_energetique_ef_electricite_pac_kwh,\
                                                                                                                agent_energetique_ef_electricite_directe_kwh=agent_energetique_ef_electricite_directe_kwh,\
                                                                                                                agent_energetique_ef_autre_kwh=agent_energetique_ef_autre_kwh,\
                                                                                                                periode_start=periode_start,\
                                                                                                                periode_end=periode_end,\
                                                                                                                ef_avant_corr_kwh_m2=ef_avant_corr_kwh_m2,\
                                                                                                                ef_objectif_pondere_kwh_m2=ef_objectif_pondere_kwh_m2,\
                                                                                                                repartition_energie_finale_partie_renovee_chauffage=repartition_energie_finale_partie_renovee_chauffage,\
                                                                                                                repartition_energie_finale_partie_renovee_ecs=repartition_energie_finale_partie_renovee_ecs,\
                                                                                                                repartition_energie_finale_partie_surelevee_chauffage=repartition_energie_finale_partie_surelevee_chauffage,\
                                                                                                                repartition_energie_finale_partie_surelevee_ecs=repartition_energie_finale_partie_surelevee_ecs,\
                                                                                                                df_meteo_tre200d0=df_meteo_tre200d0)

assert(delta_ef_visee_kwh_m2 == 113.042735042735)
# C92 → Est. ECS/ECS annuelle
assert(round(estimation_ecs_annuel, 10) == 1.0000000000)
# C93 → Est. Chauffage/Chauffage annuel prévisible
assert(round(estimation_part_chauffage_periode_sur_annuel, 10) == 0.8599805096)
# C94 → Est. EF période / EF année
assert(round(estimation_energie_finale_periode_sur_annuel, 10) == 91.3321267827)
# C95 → Est. Part ECS période comptage
assert(round(part_ecs_periode_comptage, 10) == 0.4171066572)
# C96 → Est. Part Chauffage période comptage
assert(round(part_chauffage_periode_comptage, 10) == 0.5828933428)
# C97 → correction ECS = 365/nb jour comptage
assert(round(correction_ecs, 10) == 1.0000000000)
# C98 → Energie finale indiqué par le(s) compteur(s)
assert(round(agent_energetique_ef_somme_kwh, 10) == 35226.6666666667)
# C99 → Methodo_Bww → Part de ECS en énergie finale sur la période
assert(round(methodo_b_ww_kwh, 10) == 14693.2771768406)
# C100 → Methodo_Eww
assert(round(methodo_e_ww_kwh, 10) == 9.7955181179)
# C101 → DJ ref annuels
assert(round(DJ_REF_ANNUELS, 10) == 3260.5390108363)
# C102 → DJ période comptage
assert(round(dj_periode, 10) == 2804.0000000000)
# C103 → Methodo_Bh → Part de chauffage en énergie finale sur la période
assert(round(methodo_b_h_kwh, 10) == 20533.3894898260)
# C104 → Methodo_Eh
assert(round(methodo_e_h_kwh, 10) == 15.9177169416)
# C105 → Ef,après,corr → Methodo_Eww + Methodo_Eh
assert(round(energie_finale_apres_travaux_climatiquement_corrigee_inclus_surelevation_kwh_m2, 10) == 25.7132350595)
# C106 → Part de l'énergie finale théorique dédiée à la partie rénovée (ECS+Ch.)
assert(round(repartition_energie_finale_partie_renovee_somme, 12) == 100.0)
# C107 → Ef,après,corr,rénové →Total en énergie finale (Eww+Eh) pour la partie rénovée
assert(round(energie_finale_apres_travaux_climatiquement_corrigee_renovee_kwh_m2, 10) == 25.7132350595)
# C108 → fp → facteur de pondération moyen
assert(round(facteur_ponderation_moyen, 10) == 1.2838758516)
# C109 → Methodo_Eww*fp
assert(round(methodo_e_ww_renovee_pondere_kwh_m2, 10) == 12.5762291657)
# C110 → Methodo_Eh*fp
assert(round(methodo_e_h_renovee_pondere_kwh_m2, 10) == 20.4363723943)
# C113 → Ef,après,corr,rénové*fp
assert(round(energie_finale_apres_travaux_climatiquement_corrigee_renovee_pondere_kwh_m2, 10) == 33.0126015601)
# C114 → Ef,après,corr,rénové*fp
assert(round(energie_finale_apres_travaux_climatiquement_corrigee_renovee_pondere_MJ_m2, 10) == 118.8453656163)

### Surélévation

In [398]:
# SRE
sre_renovation_m2 = 1500

# SRE pourcentage affectations
sre_pourcentage_habitat_collectif = 80.0
sre_pourcentage_habitat_individuel = 0.0
sre_pourcentage_administration = 20.0
sre_pourcentage_ecoles = 0.0
sre_pourcentage_commerce = 0.0
sre_pourcentage_restauration = 0.0
sre_pourcentage_lieux_de_rassemblement = 0.0
sre_pourcentage_hopitaux = 0.0
sre_pourcentage_industrie = 0.0
sre_pourcentage_depots = 0.0
sre_pourcentage_installations_sportives = 0.0
sre_pourcentage_piscines_couvertes = 0.0

# Agents énergétiques
agent_energetique_ef_mazout_kg = 0.0
agent_energetique_ef_mazout_litres = 500
agent_energetique_ef_mazout_kwh = 0.0
agent_energetique_ef_gaz_naturel_m3 = 0.0
agent_energetique_ef_gaz_naturel_kwh = 10000
agent_energetique_ef_bois_buches_dur_stere = 0.0
agent_energetique_ef_bois_buches_tendre_stere = 0.0
agent_energetique_ef_bois_buches_tendre_kwh = 0.0
agent_energetique_ef_pellets_m3 = 0.0
agent_energetique_ef_pellets_kg = 0.0
agent_energetique_ef_pellets_kwh = 0.0
agent_energetique_ef_plaquettes_m3 = 0.0
agent_energetique_ef_plaquettes_kwh = 0.0
agent_energetique_ef_cad_kwh = 10000
agent_energetique_ef_electricite_pac_kwh = 10000
agent_energetique_ef_electricite_directe_kwh = 0.0
agent_energetique_ef_autre_kwh = 0.0

# Météo début/fin
periode_start = datetime.date(2022,1,1)
periode_end = datetime.date(2022,12,31)

# Autres données
ef_avant_corr_kwh_m2 = 133.6111111111
ef_objectif_pondere_kwh_m2 = 40.2069603587

# Répartition énergie finale
repartition_energie_finale_partie_renovee_chauffage = 34.6026245239
repartition_energie_finale_partie_renovee_ecs = 37.7753938163
repartition_energie_finale_partie_surelevee_chauffage = 11.6735918333
repartition_energie_finale_partie_surelevee_ecs = 15.9483898265

df_meteo_tre200d0 = get_meteo_data()

delta_ef_visee_kwh_m2,\
    estimation_ecs_annuel,\
    estimation_part_chauffage_periode_sur_annuel,\
    estimation_energie_finale_periode_sur_annuel,\
    part_ecs_periode_comptage,\
    part_chauffage_periode_comptage,\
    correction_ecs,\
    agent_energetique_ef_somme_kwh,\
    methodo_b_ww_kwh,\
    methodo_e_ww_kwh,\
    DJ_REF_ANNUELS,\
    dj_periode,\
    methodo_b_h_kwh,\
    methodo_e_h_kwh,\
    energie_finale_apres_travaux_climatiquement_corrigee_inclus_surelevation_kwh_m2,\
    repartition_energie_finale_partie_renovee_somme,\
    energie_finale_apres_travaux_climatiquement_corrigee_renovee_kwh_m2,\
    facteur_ponderation_moyen,\
    methodo_e_ww_renovee_pondere_kwh_m2,\
    methodo_e_h_renovee_pondere_kwh_m2,\
    energie_finale_apres_travaux_climatiquement_corrigee_renovee_pondere_kwh_m2,\
    energie_finale_apres_travaux_climatiquement_corrigee_renovee_pondere_MJ_m2 = calcul_atteinte_objectifs_amoen(sre_renovation_m2=sre_renovation_m2,\
                                                                                                                sre_pourcentage_habitat_collectif=sre_pourcentage_habitat_collectif,\
                                                                                                                sre_pourcentage_habitat_individuel=sre_pourcentage_habitat_individuel,\
                                                                                                                sre_pourcentage_administration=sre_pourcentage_administration,\
                                                                                                                sre_pourcentage_ecoles=sre_pourcentage_ecoles,\
                                                                                                                sre_pourcentage_commerce=sre_pourcentage_commerce,\
                                                                                                                sre_pourcentage_restauration=sre_pourcentage_restauration,\
                                                                                                                sre_pourcentage_lieux_de_rassemblement=sre_pourcentage_lieux_de_rassemblement,\
                                                                                                                sre_pourcentage_hopitaux=sre_pourcentage_hopitaux,\
                                                                                                                sre_pourcentage_industrie=sre_pourcentage_industrie,\
                                                                                                                sre_pourcentage_depots=sre_pourcentage_depots,\
                                                                                                                sre_pourcentage_installations_sportives=sre_pourcentage_installations_sportives,\
                                                                                                                sre_pourcentage_piscines_couvertes=sre_pourcentage_piscines_couvertes,\
                                                                                                                agent_energetique_ef_mazout_kg=agent_energetique_ef_mazout_kg,\
                                                                                                                agent_energetique_ef_mazout_litres=agent_energetique_ef_mazout_litres,\
                                                                                                                agent_energetique_ef_mazout_kwh=agent_energetique_ef_mazout_kwh,\
                                                                                                                agent_energetique_ef_gaz_naturel_m3=agent_energetique_ef_gaz_naturel_m3,\
                                                                                                                agent_energetique_ef_gaz_naturel_kwh=agent_energetique_ef_gaz_naturel_kwh,\
                                                                                                                agent_energetique_ef_bois_buches_dur_stere=agent_energetique_ef_bois_buches_dur_stere,\
                                                                                                                agent_energetique_ef_bois_buches_tendre_stere=agent_energetique_ef_bois_buches_tendre_stere,\
                                                                                                                agent_energetique_ef_bois_buches_tendre_kwh=agent_energetique_ef_bois_buches_tendre_kwh,\
                                                                                                                agent_energetique_ef_pellets_m3=agent_energetique_ef_pellets_m3,\
                                                                                                                agent_energetique_ef_pellets_kg=agent_energetique_ef_pellets_kg,\
                                                                                                                agent_energetique_ef_pellets_kwh=agent_energetique_ef_pellets_kwh,\
                                                                                                                agent_energetique_ef_plaquettes_m3=agent_energetique_ef_plaquettes_m3,\
                                                                                                                agent_energetique_ef_plaquettes_kwh=agent_energetique_ef_plaquettes_kwh,\
                                                                                                                agent_energetique_ef_cad_kwh=agent_energetique_ef_cad_kwh,\
                                                                                                                agent_energetique_ef_electricite_pac_kwh=agent_energetique_ef_electricite_pac_kwh,\
                                                                                                                agent_energetique_ef_electricite_directe_kwh=agent_energetique_ef_electricite_directe_kwh,\
                                                                                                                agent_energetique_ef_autre_kwh=agent_energetique_ef_autre_kwh,\
                                                                                                                periode_start=periode_start,\
                                                                                                                periode_end=periode_end,\
                                                                                                                ef_avant_corr_kwh_m2=ef_avant_corr_kwh_m2,\
                                                                                                                ef_objectif_pondere_kwh_m2=ef_objectif_pondere_kwh_m2,\
                                                                                                                repartition_energie_finale_partie_renovee_chauffage=repartition_energie_finale_partie_renovee_chauffage,\
                                                                                                                repartition_energie_finale_partie_renovee_ecs=repartition_energie_finale_partie_renovee_ecs,\
                                                                                                                repartition_energie_finale_partie_surelevee_chauffage=repartition_energie_finale_partie_surelevee_chauffage,\
                                                                                                                repartition_energie_finale_partie_surelevee_ecs=repartition_energie_finale_partie_surelevee_ecs,\
                                                                                                                df_meteo_tre200d0=df_meteo_tre200d0)

assert(delta_ef_visee_kwh_m2 == 93.4041507524)
# C92 → Est. ECS/ECS annuelle
assert(round(estimation_ecs_annuel, 10) == 1.0000000000)
# C93 → Est. Chauffage/Chauffage annuel prévisible
assert(round(estimation_part_chauffage_periode_sur_annuel, 10) == 0.8599805096)
# C94 → Est. EF période / EF année
assert(round(estimation_energie_finale_periode_sur_annuel, 10) == 93.5204277665)
# C95 → Est. Part ECS période comptage
assert(round(part_ecs_periode_comptage, 10) == 0.5744604139)
# C96 → Est. Part Chauffage période comptage
assert(round(part_chauffage_periode_comptage, 10) == 0.4255395861)
# C97 → correction ECS = 365/nb jour comptage
assert(round(correction_ecs, 10) == 1.0000000000)
# C98 → Energie finale indiqué par le(s) compteur(s)
assert(round(agent_energetique_ef_somme_kwh, 10) == 35226.6666666667)
# C99 → Methodo_Bww → Part de ECS en énergie finale sur la période
assert(round(methodo_b_ww_kwh, 10) == 20236.3255136394)
# C100 → Methodo_Eww
assert(round(methodo_e_ww_kwh, 10) == 13.4908836758)
# C101 → DJ ref annuels
assert(round(DJ_REF_ANNUELS, 10) == 3260.5390108363)
# C102 → DJ période comptage
assert(round(dj_periode, 10) == 2804.0000000000)
# C103 → Methodo_Bh → Part de chauffage en énergie finale sur la période
assert(round(methodo_b_h_kwh, 10) == 14990.3411530273)
# C104 → Methodo_Eh
assert(round(methodo_e_h_kwh, 10) == 11.6206828614)
# C105 → Ef,après,corr → Methodo_Eww + Methodo_Eh
assert(round(energie_finale_apres_travaux_climatiquement_corrigee_inclus_surelevation_kwh_m2, 10) == 25.1115665372)
# C106 → Part de l'énergie finale théorique dédiée à la partie rénovée (ECS+Ch.)
assert(round(repartition_energie_finale_partie_renovee_somme, 12) == 72.3780183402)
# C107 → Ef,après,corr,rénové →Total en énergie finale (Eww+Eh) pour la partie rénovée
assert(round(energie_finale_apres_travaux_climatiquement_corrigee_renovee_kwh_m2, 10) == 18.1752542338)
# C108 → fp → facteur de pondération moyen
assert(round(facteur_ponderation_moyen, 10) == 1.2838758516)
# C109 → Methodo_Eww*fp
assert(round(methodo_e_ww_renovee_pondere_kwh_m2, 10) == 12.5363213526)
# C110 → Methodo_Eh*fp
assert(round(methodo_e_h_renovee_pondere_kwh_m2, 10) == 10.7984486553)
# C113 → Ef,après,corr,rénové*fp
assert(round(energie_finale_apres_travaux_climatiquement_corrigee_renovee_pondere_kwh_m2, 10) == 23.3347700080)
# C114 → Ef,après,corr,rénové*fp
assert(round(energie_finale_apres_travaux_climatiquement_corrigee_renovee_pondere_MJ_m2, 10) == 84.0051720287)